# Projeto 1 - Ciência dos Dados

Nome: Arthur Gomes Chieppe

Nome: João Pedro Di Jardim Marquez

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from math import *
from emoji import UNICODE_EMOJI
pd.set_option("display.max_rows", None, "display.max_columns", None)

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'PS5.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo PS5.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Tweets,Relevância:
0,@immandy4 o arquiteto do ps5 disse que ele ser...,1
1,o vídeo que eu fiz pro evento tá no youtube tb...,0
2,@chief117_com_br @vingadorbrambz enquanto o da...,0
3,possivelmente hoje sai o valor e a data de lan...,1
4,"@lealemy_ sim, mas os produtos da sony vão par...",1


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)


,Tweets,Relevância:
0,"sony nega, mas ações continuam em queda após r...",1
1,hoje tem \n\n#ps5,1
2,@prodigyrr não faça isso. ps5 ta uma completa ...,0
3,"hoje lançam o preço do ps5, tô só por saber qu...",1
4,"@1playergirl bom, nesse momento me sinto num h...",0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto trata-se do console de videogames PlayStation5. Quando extraímos os Tweets, havia uma espera coletiva acerca da livestream de lançamento do novo PlayStation 5. Assim , consideramos relevante especulações acerca do preço do novo console, bem como opiniões dos usuários sobre seu design. Ademais, consideramos algumas comparações entre o novo PlayStation e Xbox como relevantes, porém somente aquelas bem fundamentadas e intencionadas, já que consideramos irrelevantes comparações vagas e com discurso de ódio. Anúncios feitos acerca da live também foram consideradas como relevantes, com exceção de anúncios que tinham como objetivo somente a autopromoção dos usuários.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
#criamos uma função para limpar caracteres que possam atrapalhar ou enviesar o classificador:
def limpa_tweets(dataframe):
    train_limpo = dataframe.iloc[:, 0]
    ls= []
    for w in train_limpo:
        w = w.casefold()
        w = w.replace(":",'')
        w = w.replace('"','')
        w = w.replace("'",'')
        w = w.replace("(",'')
        w = w.replace(")",'')
        w = w.replace('!','')
        w = w.replace('...','')
        w = w.replace(',','')
        w = w.replace('\n','')
        w = w.replace('/','')
        w = w.replace(' a ',' ')  #Optamos por remover certos artigos e outras palavras com pouca relevância, pois 
        w = w.replace(' o ',' ')  #tendem aparecer repetidamente tanto em tweets relevantes como irrelevantes. Assim, caso não
        w = w.replace(' de ',' ') #os filtrássemos, eles teriam alta influência sobre os cálculos de probabilidade, mesmo que 
        w = w.replace(' do ',' ') #mínguo significado.
        w = w.replace(' no ',' ')
        w = w.replace(' na ',' ')
        w = w.replace(' os ',' ')
        w = w.replace(' que ',' ')
        w = w.replace(' q ',' ')
        w = w.replace(' as ',' ')
        w = w.replace(' é ',' ')
        w = w.replace(' um ',' ')
        w = w.replace(' pra ',' ')
        w = w.replace('  ',' ')
        
        w = w.replace(' - ',' ')
        w = " ".join(filter(lambda x:x[0]!='@', w.split())) #Filtramos menções a usuários do Twitter.
        w = ''.join(' ' + char+' ' if char in UNICODE_EMOJI else char for char in w) #Correção do espaçamento entre emojis.
        w = w.replace('  ',' ')
        w = w.strip()
        ls.append(w)
    dataframe['Tweets'] = ls
    return dataframe
train = limpa_tweets(train) #Substituímos o dataframe original pelo dataframe limpo.


## Classificador:





In [6]:
#Calculando Probabilidades:
total_portugues = 50000 #Consideramos uma estimativa do número de palavras diferentes comumente usadas no
                        #twitter para realizar o smoothing.
def treinamento(dataframe):
    Rele = pd.Series(dtype=object)
    Irrele = pd.Series(dtype=object)
    i = 0
    for s in dataframe['Tweets']:
        lsWords = pd.Series(s.split())
        if dataframe['Relevância:'][i] == 1: #Dependendo da classificação manual, as ocorrências depalavras contidas em um
            Rele = Rele.append(lsWords)     # tweet são adiciondas em uma lista correspodnente.
        else:
            Irrele = Irrele.append(lsWords)
        i += 1
    
    #Calcaulando a probabilidade de cada palavra ser relevante, salvando o resultado em um dicionário:
    lsRele = []
    for v in Rele.value_counts():
        lsRele.append(log10((v+1)/(total_portugues+Rele.count()))) #Utilizamos o log de base 10 das probabilidades para contornar 
    lsKeysRele = Rele.value_counts().index.tolist() #números muito pequenos ao multiplicarmos as probabilidades, para assim 
    dic_rele = {lsKeysRele[i]: lsRele[i] for i in range(len(lsKeysRele))} #evitar-mos limitações computacionais.
    
    #Calcaulando a probabilidade de cada palavra ser irrelevante, salvando o resultado em um dicionário:
    lsIrrele = []
    for v in Irrele.value_counts():
        lsIrrele.append(log10((v+1)/(total_portugues+Irrele.count())))
    lsKeysIrrele = Irrele.value_counts().index.tolist()
    dic_irrele = {lsKeysIrrele[i]: lsIrrele[i] for i in range(len(lsKeysIrrele))}
    print(dic_irrele)
    return [dic_rele, dic_irrele, Rele, Irrele]



In [7]:
#Extraindo dados de treinamento do classificador, utilizando a função "treinamento":
treino = treinamento(train)
dic_rele = treinamento(train)[0]
dic_irrele = treinamento(train)[1]
Rele = treinamento(train)[2]
Irrele = treinamento(train)[3]

#Implementando função classificadora:
def classifica(dataframe, dic_rele, dic_irrele, Rele, Irrele):
    lsClassificacao = []
    for t in dataframe['Tweets']:
        lsWord = t.split()
        lsRele = []
        lsIrrele = []
        for w in lsWord:
            if w in dic_rele:
                lsRele.append(dic_rele[w])
            if w in dic_irrele:
                lsIrrele.append(dic_irrele[w])
            else:
                lsRele.append(log10(1/(total_portugues)))
                lsIrrele.append(log10(1/(total_portugues)))
        if (log10(Rele.count()/(total_portugues+Rele.count())) + np.sum(lsRele)) <= (log10(Irrele.count()/(total_portugues+Irrele.count())) + np.sum(lsIrrele)):
            lsClassificacao.append(1)
        else:
            lsClassificacao.append(0)
    return lsClassificacao
teste_treinamento = pd.Series(classifica(train, dic_rele, dic_irrele, Rele, Irrele))
train['Classificação'] = teste_treinamento


{'ps5': -2.687058046906479, 'e': -2.7809625497748103, 'dia': -3.0972325119955286, 'bom': -3.189002885351174, 'eu': -3.243360547673767, 'vai': -3.258083804494473, 'não': -3.258083804494473, 'hoje': -3.27332377105121, '#ps5': -3.305508454422611, 'evento': -3.340270560681823, 'uma': -3.3587539663758363, 'me': -3.3587539663758363, 'da': -3.3587539663758363, 'com': -3.3587539663758363, 'se': -3.39826250765951, 'tem': -3.4417282014406, 'xbox': -3.4900328810151553, 'para': -3.4900328810151553, 'mais': -3.574353766715191, 'preço': -3.574353766715191, 'meu': -3.574353766715191, 'isso': -3.6065384500865925, 'ter': -3.6065384500865925, 'vou': -3.6413005563458043, 'ou': -3.6413005563458043, 'em': -3.679089117235204, 'sony': -3.679089117235204, 'já': -3.679089117235204, 'ser': -3.679089117235204, 'mas': -3.679089117235204, 'tá': -3.679089117235204, 'por': -3.679089117235204, 'dar': -3.720481802393429, 'ver': -3.720481802393429, 'ps5?': -3.720481802393429, 'pro': -3.766239292954104, 'aqui': -3.76623

{'ps5': -2.687058046906479, 'e': -2.7809625497748103, 'dia': -3.0972325119955286, 'bom': -3.189002885351174, 'eu': -3.243360547673767, 'vai': -3.258083804494473, 'não': -3.258083804494473, 'hoje': -3.27332377105121, '#ps5': -3.305508454422611, 'evento': -3.340270560681823, 'uma': -3.3587539663758363, 'me': -3.3587539663758363, 'da': -3.3587539663758363, 'com': -3.3587539663758363, 'se': -3.39826250765951, 'tem': -3.4417282014406, 'xbox': -3.4900328810151553, 'para': -3.4900328810151553, 'mais': -3.574353766715191, 'preço': -3.574353766715191, 'meu': -3.574353766715191, 'isso': -3.6065384500865925, 'ter': -3.6065384500865925, 'vou': -3.6413005563458043, 'ou': -3.6413005563458043, 'em': -3.679089117235204, 'sony': -3.679089117235204, 'já': -3.679089117235204, 'ser': -3.679089117235204, 'mas': -3.679089117235204, 'tá': -3.679089117235204, 'por': -3.679089117235204, 'dar': -3.720481802393429, 'ver': -3.720481802393429, 'ps5?': -3.720481802393429, 'pro': -3.766239292954104, 'aqui': -3.76623

{'ps5': -2.687058046906479, 'e': -2.7809625497748103, 'dia': -3.0972325119955286, 'bom': -3.189002885351174, 'eu': -3.243360547673767, 'vai': -3.258083804494473, 'não': -3.258083804494473, 'hoje': -3.27332377105121, '#ps5': -3.305508454422611, 'evento': -3.340270560681823, 'uma': -3.3587539663758363, 'me': -3.3587539663758363, 'da': -3.3587539663758363, 'com': -3.3587539663758363, 'se': -3.39826250765951, 'tem': -3.4417282014406, 'xbox': -3.4900328810151553, 'para': -3.4900328810151553, 'mais': -3.574353766715191, 'preço': -3.574353766715191, 'meu': -3.574353766715191, 'isso': -3.6065384500865925, 'ter': -3.6065384500865925, 'vou': -3.6413005563458043, 'ou': -3.6413005563458043, 'em': -3.679089117235204, 'sony': -3.679089117235204, 'já': -3.679089117235204, 'ser': -3.679089117235204, 'mas': -3.679089117235204, 'tá': -3.679089117235204, 'por': -3.679089117235204, 'dar': -3.720481802393429, 'ver': -3.720481802393429, 'ps5?': -3.720481802393429, 'pro': -3.766239292954104, 'aqui': -3.76623

{'ps5': -2.687058046906479, 'e': -2.7809625497748103, 'dia': -3.0972325119955286, 'bom': -3.189002885351174, 'eu': -3.243360547673767, 'vai': -3.258083804494473, 'não': -3.258083804494473, 'hoje': -3.27332377105121, '#ps5': -3.305508454422611, 'evento': -3.340270560681823, 'uma': -3.3587539663758363, 'me': -3.3587539663758363, 'da': -3.3587539663758363, 'com': -3.3587539663758363, 'se': -3.39826250765951, 'tem': -3.4417282014406, 'xbox': -3.4900328810151553, 'para': -3.4900328810151553, 'mais': -3.574353766715191, 'preço': -3.574353766715191, 'meu': -3.574353766715191, 'isso': -3.6065384500865925, 'ter': -3.6065384500865925, 'vou': -3.6413005563458043, 'ou': -3.6413005563458043, 'em': -3.679089117235204, 'sony': -3.679089117235204, 'já': -3.679089117235204, 'ser': -3.679089117235204, 'mas': -3.679089117235204, 'tá': -3.679089117235204, 'por': -3.679089117235204, 'dar': -3.720481802393429, 'ver': -3.720481802393429, 'ps5?': -3.720481802393429, 'pro': -3.766239292954104, 'aqui': -3.76623

{'ps5': -2.687058046906479, 'e': -2.7809625497748103, 'dia': -3.0972325119955286, 'bom': -3.189002885351174, 'eu': -3.243360547673767, 'vai': -3.258083804494473, 'não': -3.258083804494473, 'hoje': -3.27332377105121, '#ps5': -3.305508454422611, 'evento': -3.340270560681823, 'uma': -3.3587539663758363, 'me': -3.3587539663758363, 'da': -3.3587539663758363, 'com': -3.3587539663758363, 'se': -3.39826250765951, 'tem': -3.4417282014406, 'xbox': -3.4900328810151553, 'para': -3.4900328810151553, 'mais': -3.574353766715191, 'preço': -3.574353766715191, 'meu': -3.574353766715191, 'isso': -3.6065384500865925, 'ter': -3.6065384500865925, 'vou': -3.6413005563458043, 'ou': -3.6413005563458043, 'em': -3.679089117235204, 'sony': -3.679089117235204, 'já': -3.679089117235204, 'ser': -3.679089117235204, 'mas': -3.679089117235204, 'tá': -3.679089117235204, 'por': -3.679089117235204, 'dar': -3.720481802393429, 'ver': -3.720481802393429, 'ps5?': -3.720481802393429, 'pro': -3.766239292954104, 'aqui': -3.76623

In [18]:
#Contando ocorrências de 0 e 1 classificadas pelo algofitmo:
teste_treinamento.value_counts()

1    150
0    150
dtype: int64

In [9]:
#Comparando com as entradas manualmente digitadas:
train['Relevância:'].value_counts()

0    152
1    148
Name: Relevância:, dtype: int64

In [10]:
#Checando eficácia do classificador em se auto-classificar:
pd.crosstab(train['Relevância:'], train['Classificação'], normalize='index')

Classificação,0,1
Relevância:,,
0,0.980263,0.019737
1,0.006757,0.993243


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [11]:
test = limpa_tweets(test)
teste_teste = pd.Series(classifica(test, dic_rele, dic_irrele, Rele, Irrele))

test['Classificação'] = teste_teste


In [12]:
teste_teste.value_counts()

1    109
0     91
dtype: int64

In [13]:
test['Relevância:'].value_counts()

1    110
0     90
Name: Relevância:, dtype: int64

In [26]:
verifica_performance = pd.crosstab(test['Relevância:'], test['Classificação'], normalize='index').round(3)
verifica_performance

Classificação,0,1
Relevância:,,
0,0.556,0.444
1,0.373,0.627


In [34]:
print('Porcentagem de verdadeiros positivos (relevante): {}%' .format(verifica_performance.iloc[1,1]))
print('Porcentagem de falsos positivos (relevante): {}%' .format(verifica_performance.iloc[1,0]))
print('Porcentagem de verdadeiros negativos (irrelevante): {}%' .format(verifica_performance.iloc[0,0]))
print('Porcentagem de falsos negativos (irrelevante): {}%' .format(verifica_performance.iloc[0,1]))

Porcentagem de verdadeiros positivos (relevante): 0.627%
Porcentagem de falsos positivos (relevante): 0.373%
Porcentagem de verdadeiros negativos (irrelevante): 0.556%
Porcentagem de falsos negativos (irrelevante): 0.444%


In [17]:
#train.to_excel(r'PS5.xlsx', sheet_name='Treinamento', index = False)

___
### Concluindo

#### Análise qualitativa:

    Ao analisar-se os resultados obtidos na aplicação do classificador no DataFrame de teste, à primeira vista o resultado não demonstra-se devidamente satisfatório. De um ponto de vista técnico, no entanto, a construção e a performance do classificador atendeu aos requisitos de um Minimum Viable Product (MVP). Esse resultado relativamente insatisfatório se dá, principalmente, devido as circunstâncias nas quais extraímos nossa base de Tweets de treinamento e teste.
    
    Devido ao evento de apresentação do PS5 que ocorreria no dia em que extraímos o DataFrame, houve muita especulação acerca do preço e outras características do novo console. Naturalmente, também percebemos intensa rivalidade e discussão entre fãs de de PlayStaton e Xbox nos tweets, sendo que discussões saudáveis e informativas foram classificadas como relevantes, enquanto consideramos irrelevantes aquelas recheadas de ataques pessoais e palavras de baixo calão. Como o classificador implementado avalia a probabilidade de cada palavra individualmente, ele é incapaz de associar adjetivos como "bom","ruim", "melhor" e "pior" com as palavras as quais eles se referem. Isso impactou negativamente na capacidade do classificador de discernir a relevância de certos tweets.
    
#### Considerações sobre dupla negação e sarcasmo:

    Durante a classificação manual, os eventuais casos de sarcasmo e dupla negação foram interpratados de acordo com a intenção do autor. No entanto, como o nosso classificador Naive-Bayes trata as palavras de forma independente, ele é incapaz de naturalmente interpretar essas adversidades. Tais interpretações requerem um algoritmo muito avançado ou até mesmo um humano, visto que sarcasmo depende do contexto e da tonalidade com qual é proferido, enquanto a dupla negação só pode ser constatada ao analisarmos grupos de palavras ou frases.
    
#### Plano de expansão:

    A Sony deve continuar investindo em nosso projeto, pois à medida que avançamos com ele, aumentaremos nosso banco de dados, o que contribuiria para uma melhor precisão do classificador. Além disso, a próxima fase do projeto consistiria implementar n-gramas, de forma a considerar sequências de palavras, para assim atenuar na problema decorrente de frases com dupla negação. Também interpretaríamos palavras semelhantes, como "PS5", "PlayStation5" e "PS" em uma única palavra, de forma a simplificar cálculos a aumentar a precisão do classificador. Finalmente, seria interessante adotarmos uma medida punitiva às palavras que aparecem muito frequentemente, como o mecanismo TF-IDF, para evitar com que palavras extremamente recorrentes enviesem a classificação dos tweets.

#### Por que não posso alimentar minha base de Treinamento?
       
    Não é possível usar o classificador para obter mais dados para a tabela de treinamentos, pois isso implicaria no classificador isso apenas iria intensificar as probabilidades das palavras já calculadas e não apresentando uma nova frequência para essas.
    
#### Diferentes aplicações para o classificador Naive-Bayes:
    
    
    
    

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

Filtrar palavras específicas: https://stackoverflow.com/questions/27195769/python-remove-word-start-with-specific-character

Reconhecer emojis: https://pypi.org/project/emoji/